In [95]:
import nltk
import pandas as pd
import numpy as np
import re

from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer

In [96]:
reviews = pd.read_csv('content.csv')

In [97]:
reviews.head()

,reviewid,content
0,22703,"“Trip-hop” eventually became a ’90s punchline,..."
1,22721,"Eight years, five albums, and two EPs in, the ..."
2,22659,Minneapolis’ Uranium Club seem to revel in bei...
3,22661,Kleenex began with a crash. It transpired one ...
4,22725,It is impossible to consider a given release b...


In [98]:
len(reviews)

18393

In [99]:
info_df = pd.read_csv('reviews.csv')
genre = pd.read_csv('genres.csv')
label = pd.read_csv('labels.csv')
year_released = pd.read_csv('years.csv')

In [100]:
genre = genre.drop_duplicates(subset='reviewid', keep="first")

In [101]:
reviews = reviews.merge(genre, how='outer', on = ['reviewid'])
reviews

,reviewid,content,genre
0,22703,"“Trip-hop” eventually became a ’90s punchline,...",electronic
1,22721,"Eight years, five albums, and two EPs in, the ...",metal
2,22659,Minneapolis’ Uranium Club seem to revel in bei...,rock
3,22661,Kleenex began with a crash. It transpired one ...,rock
4,22725,It is impossible to consider a given release b...,electronic
...,...,...,...
18388,1535,The marketing guys of yer average modern megac...,jazz
18389,1341,"Well, it's been two weeks now, and I guess it'...",electronic
18390,5376,"Out of Tune is a Steve Martin album. Yes, I'l...",rock
18391,2413,"Well, kids, I just went back and re-read my re...",rock


In [102]:
info_df = info_df.drop_duplicates(subset='reviewid', keep="first")

In [103]:
reviews = reviews.merge(info_df, how='outer', on = ['reviewid'])
reviews

,reviewid,content,genre,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year
0,22703,"“Trip-hop” eventually became a ’90s punchline,...",electronic,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017
1,22721,"Eight years, five albums, and two EPs in, the ...",metal,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017
2,22659,Minneapolis’ Uranium Club seem to revel in bei...,rock,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017
3,22661,Kleenex began with a crash. It transpired one ...,rock,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017
4,22725,It is impossible to consider a given release b...,electronic,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18388,1535,The marketing guys of yer average modern megac...,jazz,let us replay!,coldcut,http://pitchfork.com/reviews/albums/1535-let-u...,8.9,0,james p. wisdom,NaN,1999-01-26,1,26,1,1999
18389,1341,"Well, it's been two weeks now, and I guess it'...",electronic,1999,cassius,http://pitchfork.com/reviews/albums/1341-1999/,4.8,0,james p. wisdom,NaN,1999-01-26,1,26,1,1999
18390,5376,"Out of Tune is a Steve Martin album. Yes, I'l...",rock,out of tune,mojave 3,http://pitchfork.com/reviews/albums/5376-out-o...,6.3,0,jason josephes,contributor,1999-01-12,1,12,1,1999
18391,2413,"Well, kids, I just went back and re-read my re...",rock,"singles breaking up, vol. 1",don caballero,http://pitchfork.com/reviews/albums/2413-singl...,7.2,0,james p. wisdom,NaN,1999-01-12,1,12,1,1999


In [104]:
label = label.drop_duplicates(subset='reviewid', keep="first")

In [105]:
reviews = reviews.merge(label, how='outer', on = ['reviewid'])
reviews

,reviewid,content,genre,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,label
0,22703,"“Trip-hop” eventually became a ’90s punchline,...",electronic,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017,virgin
1,22721,"Eight years, five albums, and two EPs in, the ...",metal,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017,hathenter
2,22659,Minneapolis’ Uranium Club seem to revel in bei...,rock,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017,static shock
3,22661,Kleenex began with a crash. It transpired one ...,rock,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017,kill rock stars
4,22725,It is impossible to consider a given release b...,electronic,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017,teklife
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18388,1535,The marketing guys of yer average modern megac...,jazz,let us replay!,coldcut,http://pitchfork.com/reviews/albums/1535-let-u...,8.9,0,james p. wisdom,NaN,1999-01-26,1,26,1,1999,ninja tune
18389,1341,"Well, it's been two weeks now, and I guess it'...",electronic,1999,cassius,http://pitchfork.com/reviews/albums/1341-1999/,4.8,0,james p. wisdom,NaN,1999-01-26,1,26,1,1999,astralwerks
18390,5376,"Out of Tune is a Steve Martin album. Yes, I'l...",rock,out of tune,mojave 3,http://pitchfork.com/reviews/albums/5376-out-o...,6.3,0,jason josephes,contributor,1999-01-12,1,12,1,1999,4ad
18391,2413,"Well, kids, I just went back and re-read my re...",rock,"singles breaking up, vol. 1",don caballero,http://pitchfork.com/reviews/albums/2413-singl...,7.2,0,james p. wisdom,NaN,1999-01-12,1,12,1,1999,touch and go


In [106]:
year_released = year_released.drop_duplicates(subset='reviewid', keep="first")

In [107]:
reviews = reviews.merge(year_released, how='outer', on = ['reviewid'])

In [108]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18393 entries, 0 to 18392
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewid        18393 non-null  int64  
 1   content         18383 non-null  object 
 2   genre           16026 non-null  object 
 3   title           18391 non-null  object 
 4   artist          18391 non-null  object 
 5   url             18393 non-null  object 
 6   score           18393 non-null  float64
 7   best_new_music  18393 non-null  int64  
 8   author          18393 non-null  object 
 9   author_type     14487 non-null  object 
 10  pub_date        18393 non-null  object 
 11  pub_weekday     18393 non-null  int64  
 12  pub_day         18393 non-null  int64  
 13  pub_month       18393 non-null  int64  
 14  pub_year        18393 non-null  int64  
 15  label           18346 non-null  object 
 16  year            17987 non-null  float64
dtypes: float64(2), int64(6), object